In [36]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import plotly.express as px
from anytree import PostOrderIter
from anytree.importer import DictImporter
import ipywidgets as widgets
import plotly.graph_objs as go
import logging


In [14]:
df = pd.read_csv('/Users/gimle/DATA-205/cap_incidents_pre_streamlit_v1.csv')

/var/folders/vt/8zqwv5_51777lk_0p42g3sdr0000gn/T/ipykernel_27406/2212335501.py:1: DtypeWarning: Columns (0,2,3,4,6,7,8,9,10,13,23,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/gimle/DATA-205/cap_incidents_pre_streamlit_v1.csv')


In [15]:
df.columns


Index(['Offence Code', 'ID', 'Dispatch_Time', 'Start_Time_crime',
       'End_Time_crime', 'Victims', 'Crime Name1', 'Crime Name2',
       'Crime Name3', 'Agency', 'Place', 'Latitude_crime', 'Longitude_crime',
       'Police District Number_crime', 'Start_Time_dispatch',
       'End_Time_dispatch', 'Priority', 'Initial Type', 'Close Type',
       'Longitude_dispatch', 'Latitude_dispatch',
       'Police District Number_dispatch', 'Arrive Cleared', 'Type Change',
       'long', 'lat', 'PRA', 'Incident_ID', 'match_status',
       'month_year_crime'],
      dtype='object')

In [16]:

df['Start_Time'] = df['Start_Time_crime'].fillna(df['Start_Time_dispatch'])
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['Police_district_Number'] = df['Police District Number_crime'].fillna(df['Police District Number_dispatch'])
df['hour'] = df['Start_Time'].dt.hour
df['month'] = df['Start_Time'].dt.month
df['day'] = df['Start_Time'].dt.day



In [17]:

df['Agency'] = df['Agency'].fillna('Unknown').astype(str)
df['match_status'] = df['match_status'].astype(str)


In [18]:
crime_dict = {
    np.nan: 'no crime report',
    'Shoplifting': 'theft from property',
    'Destruction/Damage/Vandalism of Property': 'theft from property',
    'Theft From Motor Vehicle': 'theft from auto',
    'Theft from Building': 'theft from property',
    'All Other Offenses': 'other',
    'Driving Under the Influence': 'traffic or accident',
    'Credit Card/Automatic Teller Machine Fraud': 'fraud',
    'Simple Assault': 'violent crime',
    'All other Larceny': 'theft from property',
    'Drug/Narcotic Violations': 'drug related',
    'Liquor Law Violations': 'other',
    'Burglary/Breaking and Entering': 'theft from property',
    'Identity Theft': 'fraud',
    'Forcible Fondling': 'sexual assault',
    'Motor Vehicle Theft': 'theft of auto',
    'Purse-snatching': 'theft from person',
    'False Pretenses/Swindle/Confidence Game': 'fraud',
    'Theft of Motor Vehicle Parts or Accessories': 'theft of parts',
    'Trespass of Real Property': 'other',
    'Forcible Rape': 'sexual assault',
    'Pocket/picking': 'theft from person',
    'Aggravated Assault': 'violent crime',
    'Counterfeiting/Forgery': 'other',
    'Drug Equipment Violations': 'drug related',
    'Disorderly Conduct': 'other',
    'Robbery': 'theft from person',
    'Runaway': 'runaway',
    'Forcible Sodomy': 'sexual assault',
    'Weapon Law Violations': 'other',
    'Stolen Property Offenses': 'theft from property',
    'Assisting or Promoting Prostitution': 'sex crime',
    'Prostitution': 'sex crime',
    'Murder and Nonnegligent Manslaughter': 'violent crime',
    'Family Offenses, NonViolent': 'domestic incident',
    'Impersonation': 'other',
    'From Coin/Operated Machine or Device': 'theft from property',
    'Embezzlement': 'theft from property',
    'Sexual Assault With An Object': 'sexual assault',
    'Animal Cruelty': 'violent crime',
    'Extortion/Blackmail': 'other',
    'Kidnapping/Abduction': 'abduction',
    'Intimidation': 'violent crime',
    'Arson': 'violent crime',
    'Wire Fraud': 'other',
    'Human Trafficking, Commercial Sex Acts': 'human trafficking',
    'Curfew/Loitering/Vagrancy Violations': 'other',
    'Pornography/Obscene Material': 'sex materials',
    'Statuory Rape': 'sexual assault',
    'Justifiable Homicide': 'violent crime',
    'Hacking/Computer Invasion': 'other',
    'Human Trafficking, Involuntary Servitude': 'human trafficking',
    'Welfare Fraud': 'fraud',
    'Negligent Manslaughter': 'violent crime',
    'Bribery': 'other'
}



In [ ]:
dispatch_dict = {
    'TRAFFIC/TRANSPORTATION INCIDENT': 'traffic or accident',
    'SUICIDAL PERSON/ATTEMPTED SUICIDE': 'suicide',
    'ALARMRB - RESIDENTIAL BURGLARY/INTRUSION': 'theft from property',
    'TRAFFIC VIOLATION': 'traffic or accident',
    'CDS': 'drug related', # quick google search confirms this 
    'SUSPICIOUS CIRC, PERSONS, VEHICLE': 'other',
    'WANTED PERSON, VEHICLE': 'other',
    'SUSICIOUS CIRCUMSTANCE, PERSON, VEHICLE - OCCURRED EARLIER': 'other',
    'ADMINISTRATIVE (DOCUMENT, LOST OR FOUND PROPERTY, MESSAGES,': 'not a crime',
    'ALARMRB - RESIDENTIAL HOLDUP/DUREUR/PANIC': 'violent crime',
    'ALARMRB - ALARM COMMERCIAL BURGLARY/INTRUSION': 'theft from property',
    'HARASSMENT, STALKING, THREATS': 'violent crime',
    'CHECK WELFARE': 'welfare',
    'DOMESTIC DISTURBANCE/VIOLENCE': 'domestic violence',
    'ANIMAL COMPL': 'other',
    'PEDESTRIAN STRUCK': 'traffic or accident',
    'TRAFFIC ASSIST FOR FRS': 'traffic or accident',
    'NOISE - NOISE - OTHER COMPLAINTS': 'other',
    'DOMESTIC VIOLENCE': 'domestic violence',
    'DISTURBANCE/NUISANCE': 'other',
    'TRESPASSING/UNWANTED': 'other',
    'MENTAL DISORDER': 'mental health incident',
    'ASSAULT JUST OCCURRED - ROUTINE': 'violent crime',
    'ASSIST/STANDBY': 'other',
    'THEFT/LARCENY': 'theft from property',
    'HARASSMENT, STALKING, THREATS - OCCURRED EARLIER': 'violent crime',
    'MISSING, RUNAWAY, FOUND PERSON': 'runaway',
    'STLVEHT - TRS STOLEN VEHICLE - TELEPHONE REPORTING UNIT': 'theft of auto',
    'SHOOTING': 'violent crime',
    'THEFTT - TRS THEFT/LARCENY - TELEPHONE REPORTING UNIT': 'theft from property',
    'DOMESTIC DISPUTE': 'domestic incident',
    'RESCUE WITH FRS': 'not a crime',
    'FOLLOWT-TRS / SUPPLEMENTAL INFORMATION - TELEPHONE REPORTING UNIT': 'other',
    'TRAFFIC/TRANSPORTATION INCIDENT - OCCURRED EARLIER': 'traffic or accident',
    'OVERDOSE - VIA FRS': 'drug related',
    'PARKING OFFENSE': 'traffic or accident',
    'FOLLOW UP/SUPPLEMENTAL INFORMATION': 'other',
    'THEFT/LARCENY - OCCURRED EARLIER': 'theft from property',
    'STATION RESPONSE': 'not a crime',
    'FRAUD/DECEPTION - OCCURRED EARLIER': 'fraud',
    'WEAPONS/FIREARMS': 'other',
    'DOMESTIC DISTURBANCE/VIOLENCE OCCURRED EARLIER': 'domestic violence',
    'TRAFFIC VIOLATION - OCCURRED EARLIER': 'traffic or accident',
    'ABUSE, ABANDONMENT, NEGLECT': 'domestic incident',
    'ASSIST OTHER AGENCY': 'not a crime',
    'VANDALISM, DAMAGE, MISCHIEF - OCCURRED EARLIER': 'theft from property',
    'MISC-ADMIN (DOCUMENT, LOST OR FOUND PROPERTY, MESSAGES,': 'not a crime',
    'FRAUDT-TRS FRAUD / DECEPTION - TELEPHONE REPORTING UNIT': 'fraud',
    'ASLTOS': 'violent crime',
    'ANIMAL MISC': 'other',
    'ORDNANCE - FOUND UNEXPLODED': 'other',
    'VANDALISM, DAMAGE, MISCHIEF': 'theft from property',
    'THEFT/LARCENY - HOLDING SUSPECT': 'theft from property',
    'BOX ALARM - VIA FRS': 'not a crime',
    'SEXUAL ASSAULT - OCCURRED EARLIER': 'sexual assault',
    'WORKING CODE': 'not a crime',
    'E911 DISCONNECT': 'not a crime',
    'ANIMAL RESCUE': 'not a crime',
    'THEFT/LARCENY FROM AUTO - OCCURRED EARLIER': 'theft of auto',
    'ALARMV - ALARM VEHICLE': 'not a crime',
    'PRIORITY RESPONSE TRANSPORT': 'not a crime',
    'FOLLOW UP/SUPPLEMENTAL INFORMATION - OCCURRED EARLIER': 'other',
    'EVALUATION BY MCOT': 'not a crime',
    'BURGLARY JUST OCCURRED': 'theft from property',
    'PEDESTRIAN STRUCK - OCCURRED EARLIER': 'traffic or accident',
    'ASSAULT - OCCURRED EARLIER': 'violent crime',
    'ASSAULT': 'violent crime',
    'MENTAL DISORDER - VIA FRS': 'mental health incident',
    'ALARMRB - COMMERCIAL HOLDUP/DUREUR/PANIC': 'violent crime',
    'URGENT ASSIST': 'not a crime',
    'BURGLARY': 'theft from property',
    'ANIMALVJ - ANIMAL VICIOUS JUST OCCURRED': 'other',
    'S - SUSPICIOUS SITUATION ON PATROL': 'other',
    'INDECENCY/LEWDNESS': 'other',
    'THREATOS': 'violent crime',
    'FRAUD/DECEPTION': 'fraud',
    'DISTURBANCE/NUISANCE - OCCURRED EARLIER': 'other',
    'DRIVING UNDER THE INFLUENCE': 'traffic or accident',
    'BOMB DEVICE FOUND, SUSP PACKAGE, CONTAMINATION': 'other',
    'BOMB THREAT, CBRN, CONTAMINATION': 'other',
    'THREATT - TRS HARASSMENT, STALKING, THREATS - TELEPHONE REPORTING UNIT': 'violent crime',
    'VANDALISM, DAMAGE, MISCHIEF-TRS - TELEPHONE REPORTING UNIT': 'theft from property',
    'BURGLARY - OCCURRED EARLIER': 'theft from property',
    'THEFT/LARCENY FROM AUTO': 'theft from auto',
    'DOMDOS': 'domestic incident',
    'DT - DETAIL': 'not a crime',
    'ROBBERY - OCCURRED EARLIER': 'theft from person',
    'ANIMAL VICIOUS': 'other',
    'WEAPJ': 'other',
    'ANIMAL VICIOUS - OCCURRED EARLIER': 'other',
    'WEAPSUR': 'violent crime',
    'FIREWORKS - FIREWORKS - OTHER COMPLAINTS': 'other',
    'ANIMAL ABUSE': 'violent crime',
    'INDECENCY/LEWDNESS - OCCURRED EARLIER': 'other',
    'TRESPASSING/UNWANTED - OCCURRED EARLIER': 'other',
    'ALARMU - ALARM OTHER/UNKNOWN': 'other',
    'DOMVJA': 'domestic violence',
    'WEAPONS/FIREARMS - OCCURRED EARLIER': 'violent crime',
    'HARASSOS': 'violent crime',
    'DOMESTIC DISTURBANCE/VIOLENCE - OCCURRED EARLIER': 'domestic violence',
    'LOCK OUT/IN': 'not a crime',
    'MISCELLANEOUS': 'other',
    'ALARMBB - BANK BURGLARY/INTRUSION': 'theft from property',
    'ADMINISTRATIVE (DOCUMENT, LOST OR FOUND PROPERTY, MESSAGES, - OCCURRED EARLIER': 'not a crime',
    'ASLTA': 'violent crime',
    'TD - TRAFFIC DETAIL': 'traffic or accident',
    'INV - POLICE INVESTIGATION': 'not a crime',
    'TS - TRAFFIC STOP': 'traffic or accident',
    'LOSTT-TRS ADMIN (DOCUMENT, LOST OR FOUND PROP, MESSAGES - TELEPHONE REPORTING UNIT': 'not a crime',
    'ROBBERY JUST OCCURRED': 'theft from person',
    'DISPOS': 'not a crime',
    'SEX ASSAULT': 'sexual assault',
    'HAZARDOUS MATERIAL - VIA FRS': 'other',
    'HARASSMENT, STLAKING, THREATS': 'violent crime',
    'NON-PRIORITY RESPONSE TRANSPORT': 'not a crime',
    'CARJACKING JUST OCCURRED': 'theft from person',
    'MIS - MISC ON PATROL': 'other',
    'DISPJ': 'not a crime',
    'ROBBERY': 'theft from person',
    'DOMVA': 'domestic violence',
    'ASLTJA': 'violent crime',
    'SEXASLTOS': 'sexual assault',
    'ABUSE, ABANDONMENT, NEGLECT - OCCURRED EARLIER': 'domestic incident',
    'SUICIDAL PERSON/ATTEMPTED SUICIDE - OCCURRED EARLIER': 'suicide',
    'DECEASED PERSON': 'not a crime',
    'BURGOS': 'theft from property',
    'STABBING': 'violent crime',
    'ALARMBH - BANK HOLDUP/DUREUR/PANIC': 'violent crime',
    'HARASST-TRS HARASSMENT, STALKING, THREATS - TELEPHONE REPORTING UNIT': 'violent crime',
    'MISSARDD': 'other',
    'WEAPOS': 'other',
    'ABDUCTION (KIDNAPPING) - CUSTODIAL ABDUCTION, HOSTAGE SITUAT - OCCURRED EARLIER': 'violent crime',
    'STABO - STABBING - OCCURRED EARLIER': 'violent crime',
    'CDS - OCCURRED EARLIER': 'drug related',
    'ABUSEOS': 'domestic violence',
    'TRESPJ': 'other',
    'SHOOT - SHOOTING': 'violent crime',
    'WANTED PERSON,  VEHICLE - OCCURRED EARLIER': 'other',
    'SEXUAL ASSAULT': 'sexual assault',
    'ABDUCTION (KIDNAPPING) - CUSTODIAL ABDUCTION, HOSTAGE SITUAT': 'domestic violence',
    'CHECK THE WELFARE - OCCURRED EARLIER': 'welfare',
    'SS - SUBJECT STOP': 'not a crime',
    'ANI - ANIMAL COMPL ON PATROL': 'other',
    'DOMVOS': 'domestic violence',
    'SHOOTO - SHOOTING - OCCURRED EARLIER': 'violent crime',
    'NOISEO - NOISE OCCURRED EARLIER - OTHER COMPLAINTS - OCCURRED EARLIER': 'other',
    'STALKOS': 'violent crime',
    'ADMINISTRATIVE (DOCUMENT, LOST OR FOUND PROP, MESSAGES - OCCURRED EARLIER': 'not a crime',
    'PROSTITUTION': 'sex crime',
    'TRESPOS': 'other',
    'CRYWOLF INTERFACE INCIDENT TYPE': 'other',
    'NEGLECTOS': 'domestic incident',
    'TRF - TRAFFIC PROBLEM ON PATROL': 'traffic or accident',
    'ANIMAL ABUSE - OCCURRED EARLIER': 'other',
    'STAB - STABBING': 'violent crime',
    'CAR JACKING - OCCURRED EARLIER': 'theft from person',
    'ANIMAL MISC - OCCURRED EARLIER': 'other',
    'SIGNAL3': 'other',
    'TRAIN COLLISION - VIA FRS': 'traffic or accident',
    'HARRASSOS': 'violent crime',
    'ROBOS': 'theft from person',
    'DEERP - DEER ON PATROL': 'other',
    'NEGLECTJ': 'domestic incident',
    'EXPARTE SERVICE': 'not a crime',
    'ACCELERATOR STUCK': 'traffic or accident',
    'FRAUDJ': 'fraud',
    'GH - HOSPITAL GUARD DETAIL': 'not a crime',
    'ACTIVE ASSAILANT/SHOOTER': 'violent crime',
    'T - TRANSPORT': 'traffic or accident',
    'CAR JACKING': 'theft from person',
    'DECEASED PERSON - SUSPICIOUS': 'other',
    'SEXASLTJA': 'sexual assault',
    'TRAFFIC/TRANSPORTATION  INCIDENT - OCCURRED EARLIER': 'traffic or accident',
    'ODJOB - OFF DUTY JOB': 'not a crime',
    'HUNT - HUNTING - ILLEGAL': 'other',
    'DRIVING UNDER THE INFLUENCE - OCCURRED EARLIER': 'traffic or accident',
    'ABUSEJ': 'domestic violence',
    '10 SIGNAL ALARM': 'other',
    'THEFTFAT - TRS / SUPPLEMENTAL INFORMATION - TELEPHONE REPORTING UNIT': 'theft from property',
    'WS - WARRANT SERVICE': 'not a crime',
    'TP - TRAFFIC PURSUIT': 'traffic or accident',
    'RAPEO - OCCURRED EARLIER': 'sexual assault',
    'ANIMAL RESCUE - OCCURRED EARLIER': 'not a crime',
    'VANDP - VAND ON PATROL': 'other',
    'SHOTSOS': 'violent crime',
    'SEXASLTA': 'sexual assault',
    'EVICTION': 'other',
    'ASSAULT-TRS - TELEPHONE REPORTING UNIT': 'violent crime',
    'OC - OUT OF COUNTY': 'not a crime',
    'THEFTFAJ': 'theft from property',
    'CALLER IN DANGER': 'violent crime',
    'PLANE DOWN - VIA FRS': 'not a crime',
    'WANTED-TRS - TELEPHONE REPORTING UNIT': 'other',
    'PROSTITUTION - OCCURRED EARLIER': 'sex crime',
    'TRESPASSING-TRS - TELEPHONE REPORTING UNIT': 'other',
    'EMERGENCY PETITION SERVICE': 'not a crime',
    'ADMINISTRATIVE (DOCUMENT, LOST OR FOUND PROP, MESSAGES-TRS - OCCURRED EARLIER': 'not a crime',
    'TC - TRAFFIC COLLISION ON PATROL': 'traffic or accident',
    'DECEASED PERSON - OCCURRED EARLIER': 'not a crime',
    'THEFT/LACENY FROM AUTO': 'theft from auto',
    'DC - DISORD COND ON PATROL': 'not a crime',
    'SEXP - SEX OFFENSE ON PATROL': 'sex crime',
    'RP - RECOVERED PROPERTY ON PATROL': 'not a crime',
    'THE - THEFT ON PATROL': 'theft from property',
    'ALARMT': 'not a crime',
    'ALARMF - FARS': 'not a crime',
    'OVERDOSE - VIA FRS - OCCURRED EARLIER': 'drug related',
    'STABOS': 'violent crime',
    'ALRM - ALARM ON PATROL': 'other',
    'SUM - SUMMONS SERVICE': 'not a crime',
    'RAPEOS': 'sexual assault',
    'HOMEDTN-HOME DETENTION': 'not a crime',
    np.nan : 'no dispatch',
    'CDSP - CDS ON PATROL': 'not a crime',
    'WR - WRITING REPORT': 'not a crime',
    'ROBP - ROBBERY ON PATROL': 'theft from person',
    'ATT - ATTACHMENT ON PATROL': 'not a crime',
    'REFFERING CALLERS TO  HEALTH CARE PROVIDERS': 'not a crime',
    'AUTO THEFT ON PATROL': 'theft of auto',
    'RAPEJ': 'sexual assault',
    'SHOOTOS': 'violent crime',
    'RAPEJA': 'sexual assault',
    'RAPE': 'sexual assault'
}

In [20]:
df['categories_crime'] = df['Crime Name2'].map(crime_dict)

In [21]:
df['categories_dispatch'] = df['Close Type'].map(dispatch_dict)

In [30]:
df['category'] = df['categories_crime'].fillna(df['categories_dispatch'])

In [ ]:
df_treemap = df.drop(['Offence Code', 'ID', 
       'Crime Name3', 'Latitude_crime', 'Longitude_crime',
       'Police District Number_crime', 'Start_Time_dispatch',
       'End_Time_dispatch', 'Initial Type', 'Close Type',
       'Longitude_dispatch', 'Latitude_dispatch',
       'Police District Number_dispatch', 'Arrive Cleared', 'Type Change',
       'long', 'lat', 'PRA', 'Incident_ID', 'month_year_crime','Dispatch_Time', 'Start_Time_crime', 'End_Time_crime','categories_crime', 'categories_dispatch'
       ], axis=1)
df_treemap.columns

Index(['Victims', 'Crime Name1', 'Crime Name2', 'Agency', 'Place', 'Priority',
       'match_status', 'Start_Time', 'Police_district_Number', 'hour', 'month',
       'day', 'categories_crime', 'categories_dispatch'],
      dtype='object')

In [24]:
df_treemap.isna().sum()

Victims                   1175849
Crime Name1               1175849
Crime Name2               1175849
Agency                          0
Place                     1184313
Priority                    23498
match_status                    0
Start_Time                      0
Police_district_Number          0
hour                            0
month                           0
day                             0
categories_crime                2
categories_dispatch         11825
dtype: int64

## Treemap

In [33]:
labels = {'Police_district_Number': 'Police District',
          'categories_crime': 'Crime Category',
          'categories_dispatch': 'Dispatch Category',
          'hour': 'Hour of Day',
          'month': 'Month',
          'day': 'Day of Month',
          'Agency': 'Agency',
          'match_status': 'Report Match',
          'category': 'Category'}

In [37]:
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)


def treemap(df_treemap, selection):
    try:
# I need df_treemap, selection
# Group data by match_status and my selection
        grouped_data = df_treemap.groupby(['match_status', '{selection}']).size().reset_index(name='count')

        labels = []
        parents = []
        values = []

        for parent in grouped_data['match_status'].unique():
            labels.append(parent)
            parents.append('Montgomery County')
            values.append(grouped_data[grouped_data['match_status'] == parent]['count'].sum())
    
            children = grouped_data[grouped_data['match_status'] == parent]
            for _, row in children.iterrows():
                labels.append(f"District {row['{selection}']}")
                parents.append(parent)
                values.append(row['count'])

        treemap = go.Figure(go.Treemap(
            labels=labels,
            parents=parents,
            values=values,
            textinfo="value",
            branchvalues="total"
        ))

        treemap.update_layout(
            title='Treemap of the number of Dispatches and Crime Reports with Matching ID`s by ',
            width=900,
            height=700,
            treemapcolorway=["#636efa","#" "#EF553B", "#00cc96"], # fix colour palette
            margin=dict(t=50, l=25, r=25, b=25)
        )

        return treemap
    except Exception as e: 
        logger.error('error in treemap function', exc_info=True)
        raise


In [ ]:
# creates slider
def slider(sliderVal):

   treemap = plotly.graph_objs.Figure()
   treemap.add_trace(plotly.graph_objs.Treemap(
      labels = df[df[‘level’]<sliderVal][‘name’],
))

## Customization of treemap

In [ ]:
widgets.interact(update, sliderVal = (0, 3))

### Slider

In [ ]:
# Updates size and colour
fig.update_traces(root_color= "#f1f1f1", branchvalues ='total', width = 500, height = 500)